In [119]:
import pandas as pd
import numpy as np
import datetime as dt

In [120]:
df = pd.read_csv('/home/roman/git_projects/RFM_analysis/bonuscheques_202312252155.csv')
df.head()

,datetime,shop,card,bonus_earned,bonus_spent,summ,summ_with_disc,doc_id
0,2021-07-13 12:56:09.000,Аптека 2,2000200195023,51,0,3400,3400,15#2002741#65938#2_29
1,2021-07-30 10:42:00.000,Аптека 2,2000200193494,57,0,747,747,15#2002972#65955#2_5
2,2021-10-11 12:55:23.000,Аптека 2,2000200199106,92,253,3077,3077,15#2004060#66028#2_29
3,2021-10-14 14:48:56.000,Аптека 2,2000200168768,1,0,54,54,15#2004107#66031#2_57
4,2021-10-20 11:09:39.000,Аптека 2,2000200226314,101,0,1733,1733,15#2004192#66037#2_16


In [121]:
df.shape

(38486, 8)

In [122]:
df.dtypes

datetime          object
shop              object
card              object
bonus_earned       int64
bonus_spent        int64
summ               int64
summ_with_disc     int64
doc_id            object
dtype: object

In [123]:
df.isna().sum()

datetime          0
shop              0
card              0
bonus_earned      0
bonus_spent       0
summ              0
summ_with_disc    0
doc_id            0
dtype: int64

In [124]:
df['datetime'] = df['datetime'].apply(pd.to_datetime)
snapshot_date = df['datetime'].max()
df_RFM = df.groupby('card').agg({
    'datetime': lambda x: (snapshot_date - x.max()).days,
    'card': 'nunique',
    'summ_with_disc': 'sum'
})

In [125]:
df_RFM

,datetime,card,summ_with_disc
card,,,
001848d0-e56a-412e-be0a-423dfa0e564f,26,1,4391
001cb0a0-e62e-47a9-995e-599bdc7f8750,27,1,5887
00572591-c154-4b9f-83e4-3ce5b00bf9c8,244,1,557
00625996-f49f-4863-8132-40d08e99b2da,56,1,4755
00703805-f16d-4b83-b7ad-affb2736aa0d,15,1,1555
...,...,...,...
ff9695c5-6e8c-4446-adc6-944b2731b8f7,172,1,559
ffbf9d44-679e-412a-9927-ac6ce80c49a2,286,1,539
ffd22196-662c-4266-b3be-024a42cad78b,4,1,2052


In [126]:
df_RFM.rename(columns={
    'datetime': 'Recency',
    'card': 'Frequency',
    'summ_with_disc': 'Monetary'
}, inplace=True)

In [127]:
df_RFM

,Recency,Frequency,Monetary
card,,,
001848d0-e56a-412e-be0a-423dfa0e564f,26,1,4391
001cb0a0-e62e-47a9-995e-599bdc7f8750,27,1,5887
00572591-c154-4b9f-83e4-3ce5b00bf9c8,244,1,557
00625996-f49f-4863-8132-40d08e99b2da,56,1,4755
00703805-f16d-4b83-b7ad-affb2736aa0d,15,1,1555
...,...,...,...
ff9695c5-6e8c-4446-adc6-944b2731b8f7,172,1,559
ffbf9d44-679e-412a-9927-ac6ce80c49a2,286,1,539
ffd22196-662c-4266-b3be-024a42cad78b,4,1,2052
